In [1]:
import numpy as np
import linecache
from pathlib import Path
from tqdm import tqdm

In [2]:
import sys

p = Path('.').resolve()
sys.path.append(str(p.parent))

In [3]:
from utils.faiss_utils import train_search
from utils.data_utils import *

Loading faiss with AVX2 support.


In [4]:
def load_XY(basename):
    """
    Load embeddings (X) and possibly the
    labels (Y) of the graph {basename}.
    """
    model_path = Path("/data/models") / basename
    print("Loading data..")
    X, Y = load_data(model_path)
    classes = len(np.unique(Y))
    print("X shape: {}".format(X.shape))
    return X, Y

In [11]:
basename = "itwiki-2013"

nodes = nodes_from_ascii(basename)
embeddings = load_XY(basename)
X = embeddings[0]
assert len(X) == len(nodes)

1016867 vertices
reading..


100%|██████████| 1016867/1016867 [00:11<00:00, 90387.66it/s]


Found 8215 singleton nodes
Loading data..
Labels not defined
Labels not defined
Labels not defined
Labels not defined
Labels not defined
Labels not defined
Labels not defined
Labels not defined
Labels not defined
Labels not defined
X shape: (1016179, 64)


AssertionError: 

In [12]:
len(X)

1016179

In [13]:
len(nodes)

1016867

In [8]:
nodes[:10]

[[1, 3, 8, 10, 255748, 353889, 436291, 643920],
 [0, 3, 4, 5, 10, 255748, 353889, 436291],
 [0, 201, 193566, 193671, 193694, 193861, 275965, 353889, 620194],
 [0, 1, 4, 8, 9, 255748, 353889, 436291],
 [1,
  3,
  5,
  6,
  9,
  103786,
  103788,
  255748,
  353889,
  436291,
  515060,
  515063,
  842247],
 [1, 4, 6, 7, 10, 255748, 353889, 436291],
 [4, 5, 7, 273, 276, 48105, 103786, 255748, 353889, 397214, 436291, 515158],
 [5, 6, 10, 270, 273, 255748, 353889, 391617, 436291, 515152, 515158],
 [0, 3, 9, 255748, 353889, 436291, 643919, 643920],
 [3, 4, 8, 255748, 353889, 436291, 643919, 643928, 643929, 842247]]

In [14]:
perm[:10]

array([186403, 675690, 762181, 268272, 998819,  92536,  93525, 360313,
       441827, 962691])

In [10]:
ent_list = get_entities_list(basename)
perm = np.argsort(ent_list)
X = X[perm]
ind = train_search(X)

Index trained: True
Index total: 1016179


In [18]:
from measure_map import map_score


score, _, _ = map_score(X[:100], nodes[:100], ind)

In [19]:
score

0.0

In [24]:
f = "/data/graphs/itwiki-2013/itwiki-2013.ids"
print(linecache.getline(f, 1))
for i in nodes[0]:
    print(linecache.getline(f, i + 1))

Cabra de Mora

Valbona

Formiche Alto

El Castellar

Mora de Rubielos

Comunità autonome della Spagna

Spagna

Aragona

Alcalá de la Selva



In [26]:
_, I = ind.search(X[0].reshape(1, -1), 10)

In [28]:
for i in I.flatten():
    print(linecache.getline(f, i + 1))

Platinum Dunes

Orzale

Australian Men's Hardcourt Championships 1997 - Doppio

Phanias

Generazione perfetta

La figlia del mio capo

Ossario italiano di Crimea

Atmosfera tecnica

Movie 43

Diga di Egschi



In [29]:
embeddings = load_XY(basename)

Loading data..
Labels not defined
Labels not defined
Labels not defined
Labels not defined
Labels not defined
Labels not defined
Labels not defined
Labels not defined
Labels not defined
Labels not defined
X shape: (1016179, 64)


In [30]:
X = embeddings[0]

In [31]:
X.shape

(1016179, 64)

In [32]:
ind = train_search(X)

Index trained: True
Index total: 1016179


In [33]:
_, I = ind.search(X[0].reshape(1, -1), 10)

In [55]:
print(linecache.getline(f, ent_list[0] + 1))
for i in I.flatten():
    line = ent_list[i] + 1
    print(linecache.getline(f, line))

Platinum Dunes

Platinum Dunes

Generazione perfetta

La figlia del mio capo

Movie 43

Big Mama - Tale padre, tale figlio

The Roommate - Il terrore ti dorme accanto

Il gatto... e il cappello matto

Gigolò per sbaglio

After.Life

David Twohy



In [37]:
model_path = Path("/data/models") / basename
with (model_path / "entity_names_link_0.json").open() as tf:
    entities_list = json.load(tf)
hf_path = list(model_path.glob("embeddings_link_0*.h5"))[0]
hf = h5py.File(hf_path)
x = hf["embeddings"][:]
x.shape

(101618, 64)

In [43]:
idx = train_search(x)

Index trained: True
Index total: 101618


In [62]:
_, I = idx.search(x[0].reshape(1, -1), 20)

In [63]:
I

array([[    0, 85607, 51742, 41295, 79546, 33805, 68130, 31444, 99764,
        68186, 96999, 65176,  5788,   761, 65523, 76619, 88221, 85566,
        76691, 96717]])

In [64]:
for i in I.flatten():
    line = ent_list[i] + 1
    print(linecache.getline(f, line))

Platinum Dunes

Generazione perfetta

La figlia del mio capo

Movie 43

Big Mama - Tale padre, tale figlio

The Roommate - Il terrore ti dorme accanto

Il gatto... e il cappello matto

Gigolò per sbaglio

After.Life

David Twohy

Ti presento Bill

The Boondock Saints 2 - Il giorno di Ognissanti

Chestnut - Un eroe a quattro zampe

Accerchiato

Best Men - Amici per la pelle

The Weinstein Company

MTV Movie Awards 2002

One for the Money

Cenerentola e gli 007 nani

Ho rapito Sinatra



In [61]:
temp = ent_list[0]
print(linecache.getline(f, temp + 1))
for i in nodes[temp]:
    print(linecache.getline(f, i + 1))

Platinum Dunes

2013

Non aprite quella porta (film 2003)

Non aprite quella porta - L'inizio

Suda e cresci

Il mai nato

Michael Bay

Film

Los Angeles

2001

Amityville Horror (film 2005)

Novembre

The Horsemen

The Hitcher (film 2007)

Nightmare (film 2010)

Venerdì 13 (film 2009)

